학습된 모델을 불러와서 카톡대화를 넣으면\
나타난 감정의 순위를 보여준다

In [3]:
import re
import pickle
import random
import numpy as np

from soynlp.normalizer import repeat_normalize

import tensorflow as tf
from transformers import BertTokenizerFast
from transformers import TFAlbertForSequenceClassification


In [4]:
# 저장된 모델 불러오기
loaded_model = TFAlbertForSequenceClassification.from_pretrained('./albert-kor-base')

    
# 피클 파일에서 BertTokenizerFast 객체 로드
with open('./albert_tokenizer.pkl', "rb") as f:
    tokenizer = pickle.load(f)

All model checkpoint layers were used when initializing TFAlbertForSequenceClassification.

All the layers of TFAlbertForSequenceClassification were initialized from the model checkpoint at ./albert-kor-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertForSequenceClassification for predictions without further training.


In [8]:
# 카톡대화 불러와서 정제(정규화코드)하는 함수
def get_from_txt(txt):
    data= open(txt,"r", encoding='utf-8').read().split('\n')
    sentences=[]
    for line in data:
        pattern = r'\[(.*?)\]\s+\[(.*?)\]\s+(.+)'
        match = re.match(pattern, line)
        if match:
            name = match.group(1)  # 첫 번째 대괄호 안의 단어 추출
            time = match.group(2)  # 두 번째 대괄호 안의 단어 추출
            content = match.group(3)  # 대괄호 뒤의 내용 추출
            # print(name, time, content)
            temp=[name,time,content]
            sentences.append(temp)    
    return sentences

#참여자 뽑기
def get_user(katok):
    # 중복을 제거하고 참여자 리스트 생성
    user_names = list(set([i[0] for i in katok]))
    return user_names

#두명의 대화를 [[찬란카톡],[하영카톡]]형태로 얻는 함수
def get_convers(user_names):
    received_texts = [
        [j[2] for j in get_from_txt('sample.txt') if i == j[0]]  ##
        for i in user_names]
    # 이모티콘, 사진, 샵검색 제거 
    exclusion_list = ['사진', '이모티콘', '샵검색:']
    convers_texts = [
    [str(j) for j in i if all(exclusion not in j for exclusion in exclusion_list)]
    for i in received_texts]
    return convers_texts

# 반복,띄어쓰기 정제(Cleaning) 함수
def clean_korean_text(convers_texts):
    convers_texts = re.sub(r'http\S+', '', convers_texts)  #링크를 지운다
    convers_texts = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\s.!?~^;]', ' ', convers_texts)
    convers_texts = repeat_normalize(convers_texts, num_repeats=2)
    clean_texts = re.sub(r'\s+', ' ', convers_texts).strip()
    return clean_texts

# 입력문장을 정제하고 공백대화를 제거함(https도 같이 없어짐)
# [[찬란카톡],[하영카톡]]가 들어와서 정제하는 함수
def get_clean(convers_texts):
    temp = []
    for convers_text in convers_texts:
        temp.append([ clean_korean_text(i) for i in convers_text if clean_korean_text(i) !='' ]) ##
    clean_texts = temp
    del temp
    return clean_texts

#워드피스 토큰화 적용
def tokenize(clean_texts):
    temp = []
    for clean_text in clean_texts: 
        clean_text = tokenizer(clean_text, truncation=True, padding='max_length', max_length=64,return_token_type_ids= False)
        temp.append(clean_text)
    tokenized_texts = temp
    del temp
    return tokenized_texts

#모델예측 
#함수에 입력되는 형태 [[찬란토큰화카톡],[하영토큰화카톡]] , user_names['김찬란','김하영']
#                       
#출력되는 형태 : ['김찬란',[12, 23, 45, 12, 23, 45, 45],['공포', '놀람', '분노','슬픔', '중립', '행복', '혐오']]
#               ['김찬란', [공포문장들],[슬픔문장들],,,,,]
#              ,['김하영',[12, 23, 45, 12, 23, 45, 45],['공포', '놀람', '분노','슬픔', '중립', '행복', '혐오']]
#               ['김하영', [공포문장들],[슬픔문장들],,,,,]

def get_prediction(tokenized_texts, loaded_model, user_names):
    predicted = []
    for order,tokenized_text in enumerate(tokenized_texts):
        predictions=loaded_model.predict([
            np.array(tokenized_text['input_ids']),
            np.array(tokenized_text['attention_mask'])
            ])
        counts = [0, 0, 0, 0, 0, 0, 0]
        cnt=-1
        for prediction in predictions[0]:
            indices = np.around(tf.nn.softmax(prediction)).astype(int)
            cnt+=1
            #여기서 배열이 다 0이면 컨티뉴 해버린다.
            if len(np.where(indices == 1)[0]) == 0:
                continue
            index = np.where(indices == 1)[0][0]
            counts[index] += 1
        name=user_names[order]
        counts_emo_rank = sorted(counts, reverse=True)
        #순위 정렬하기
        emo_list = ['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오']
        index_list = list(range(len(counts)))
        emo_index_rank = sorted(index_list, key = lambda x: counts[x], reverse=True)
        emo_rank = []
        for i in emo_index_rank:
            emo_rank.append(emo_list[i])
        # [['김찬란',[12, 23, 45, 65, 34, 65, 34],[감정순위대로]], ['김하영',[12, 23, 45, 65, 34, 65, 34],[감정순위대로]]]
        predicted.append([name, counts_emo_rank, emo_rank])
    return predicted



In [9]:
#      ['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오']
#순서대로  0       1       2      3       4       5       6 

katok =get_from_txt('./sample.txt')
user_names = get_user(katok)
convers_texts = get_convers(user_names)
clean_texts = get_clean(convers_texts) #안에 clean_korean_text()포함
tokenized_texts = tokenize(clean_texts)
get_prediction(tokenized_texts, loaded_model, user_names)





6/6 [==============================] - 17s 3s/step


[['김찬란',
  [39, 31, 26, 26, 16, 15, 15],
  ['놀람', '행복', '슬픔', '중립', '혐오', '공포', '분노']],
 ['김하영',
  [41, 29, 28, 22, 21, 16, 8],
  ['행복', '놀람', '슬픔', '중립', '분노', '혐오', '공포']]]

In [41]:

user_names = ['김찬란', '김하영']
convers_texts = [['너는 어쩔려고 인생 그렇게 사냐?','욕하지 마라 시발롬아 뒤지고 싶나 손에 든거부터 내려놓고 해달라','그래 돼지년아'
  ,'씹돼지새끼야','시뱅아','개잡년아','디질래','야이 새끼야 니얼굴에 공부도 못하면 답없다','ㅋㅋㅋㅋㅋ'],
 ['남이사 인생 어떻게 살든말든 니가 뭔상관인데 개새끼야','너 근데 왜 중간중간에 자꾸 돼지 넣어?','병신이냐?','그게 자극이냐?',
  '사람놀리는거지 뱅쉰아?','야이 애미출타한 씨팔년아','못한다고 씨발 좆같은년아','됐지?']]
clean_texts = get_clean(convers_texts)
tokenized_texts = tokenize(clean_texts)

predicted_1 = []
# predicted_2 = []
for order,tokenized_text in enumerate(tokenized_texts):
    predictions=loaded_model.predict([
        np.array(tokenized_text['input_ids']),
        np.array(tokenized_text['attention_mask'])
        ])
    counts = [0, 0, 0, 0, 0, 0, 0]
    index_of_fear_texts= []
    index_of_surprised_texts= []
    index_of_anger_texts= []
    # index_of_sadness_texts= []
    # index_of_neutral_texts= []
    # index_of_happy_texts= []
    # index_of_aversion_texts= []  #혐오
    cnt=-1
    for prediction in predictions[0]:
        indices = np.around(tf.nn.softmax(prediction)).astype(int)
        cnt+=1
        #여기서 배열이 다 0이면 컨티뉴 해버린다.
        if len(np.where(indices == 1)[0]) == 0:
            continue
        index = np.where(indices == 1)[0][0]
        counts[index] += 1
        #강부,강긍의 인덱스를저장한다.
        if index == 0:
            index_of_fear_texts.append(cnt)
        if index == 1:
            index_of_surprised_texts.append(cnt)
        if index == 2:
            index_of_anger_texts.append(cnt)

            
    name=user_names[order]
    fear_texts = [ convers_texts[order][i] for i in index_of_fear_texts]
    surprised_texts = [ convers_texts[order][i] for i in index_of_surprised_texts]
    anger_texts = [ convers_texts[order][i] for i in index_of_anger_texts]
    
    #순위 정렬하기
    emo_list = ['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오']
    index_list = list(range(len(counts)))
    emo_index_rank = sorted(index_list, key = lambda x: counts[x], reverse=True)
    emo_rank = []
    for i in emo_index_rank:
        emo_rank.append(emo_list[i])
    counts_emo_rank = sorted(counts, reverse=True)
    

    #['김찬란',[12, 23, 45, 65, 34, 65, 34],[감정순위대로]]
    predicted_1.append([name, counts_emo_rank, emo_rank])
    # predicted_2.append([name, fear_texts, surprised_texts, anger_texts])
    
predicted_1


1/1 [==============================] - 1s 817ms/step


[['김찬란', [6, 2, 1, 0, 0, 0, 0], ['분노', '행복', '놀람', '공포', '슬픔', '중립', '혐오']],
 ['김하영', [4, 3, 1, 0, 0, 0, 0], ['분노', '놀람', '혐오', '공포', '슬픔', '중립', '행복']]]